Koble informasjon fra nærmeste basert på geografi.
I dette eksempelet vises hvordan man kobler informasjon fra holdeplasser til adresser med bosatte.
I tillegg legges det til en kolonne med avstanden.

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import shutil

pd.set_option('display.float_format', lambda x: '%.5f' % x)
gpd.options.use_pygeos = True

In [7]:
# Lese inn adressebosatte
bosatte = gpd.read_file("/ssb/x_disk/Felles/GIS-ressurssenter/LettvektsGIS/Demostrasjonsdata/Demodata_kart.gdb", layer = "Adressepkt_demo")

# Lese in holdeplasser
stops = gpd.read_file("/ssb/x_disk/Felles/GIS-ressurssenter/LettvektsGIS/Demostrasjonsdata/Demodata_kart.gdb", layer = "Bussholdeplasspkt_demo")
# Velge ut med minst 1 times avgang
stops1time = stops[stops["DepFreq"].isin(['30 min', '60 min'])]


In [6]:
bosatte.head()

,GATENR_GAA,HUSNR_BRUK,BOKSTAV_FE,M0_5,K0_5,M6_15,K6_15,M16_19,K16_19,M20_66,K20_66,M67MER,K67MER,population,FYLKE,NAVN,KOMMUNENR,POINT_X,POINT_Y,geometry
0,02375,0304,0001,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,3.00000,34,Eidskog,3416,323358.06390,6643010.98460,POINT (323358.064 6643010.985)
1,01266,0177,0001,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,3.00000,34,Eidskog,3416,323364.42050,6642778.09620,POINT (323364.421 6642778.096)
2,03877,0264,0001,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,3.00000,34,Eidskog,3416,323435.23870,6642946.58930,POINT (323435.239 6642946.589)
3,02050,0027,0001,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,3.00000,34,Eidskog,3416,323408.88730,6643028.79050,POINT (323408.887 6643028.791)
4,04283,0442,0001,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,1.00000,3.00000,34,Eidskog,3416,323420.36460,6642932.81140,POINT (323420.365 6642932.811)


In [21]:
stops.head()

,stop_id,stop_name,stop_lat,stop_lon,vehicle_ty,wheelchair,avg,kjoretoy,AvgTime,NAVN,KOMMUNENR,DepFreq,geometry
0,NSR:Quay:15587,Kulblik,59.90580,11.98510,700,2,6.00000,buss,0.42857,Eidskog,3416,30 min,POINT (331396.713 6644759.751)
1,NSR:Quay:15583,Klanderudvegen,59.91220,11.98440,700,2,9.00000,buss,0.64286,Eidskog,3416,30 min,POINT (331390.050 6645473.787)
2,NSR:Quay:15581,Sæterstua,59.92250,11.98900,700,2,9.00000,buss,0.64286,Eidskog,3416,30 min,POINT (331699.303 6646608.362)
3,NSR:Quay:15727,Enger,59.92940,12.21290,700,2,1.00000,buss,0.07143,Eidskog,3416,30 min,POINT (344241.498 6646828.128)
4,NSR:Quay:15579,Eidskog montessoriskole,59.92670,11.99260,700,2,9.00000,buss,0.64286,Eidskog,3416,30 min,POINT (331921.685 6647066.631)


In [3]:
# Beregner naermeste holdeplass
bosatte_mstops = bosatte.sjoin_nearest(stops, how="left",distance_col="avstand")

In [19]:
# Skriver til geodatasett for visuell inspeksjon 
bosatte_mstops.to_file("Demodata.gpkg", layer='bosatte_mholdeplassinfo', driver="GPKG")
shutil.copyfile("Demodata.gpkg", "/ssb/x_disk/A400/426/areal/prosjekter/Test_geodata/Demodata.gpkg")


'/ssb/x_disk/A400/426/areal/prosjekter/Test_geodata/Demodata.gpkg'

In [1]:
# antall rader
#len(bosatte_mstops.index)
#bosatte_mstops.columns
#len(bosatte)

In [52]:
# Fjerner dubletter, dvs der det er akkurat like langt til flere stop
bosatte_mstop2 = bosatte_mstops.drop_duplicates(subset=['KOMMUNENR_left','HUSNR_BRUK', 'GATENR_GAA','BOKSTAV_FE','geometry'])
#len(bosatte_mstop2.index)

6889

In [69]:
# Klassifiserer
bosatte_mstop2["AvstKlasse"] = np.where(bosatte_mstop2["avstand"] > 500, "Mer enn 500 m", "Innen 500 m") 

In [75]:
# Summerer 
bosatte_innen_holdeplass = bosatte_mstop2.groupby(['KOMMUNENR_left','AvstKlasse'], as_index=False).agg({'population': 'sum'})

In [76]:
bosatte_innen_holdeplass.head()

,KOMMUNENR_left,AvstKlasse,population
0,3401,Innen 500 m,8115.00000
1,3401,Mer enn 500 m,489.00000
2,3414,Innen 500 m,2169.00000
3,3414,Mer enn 500 m,402.00000
4,3415,Innen 500 m,2340.00000


In [78]:
# Pivotere
bosatte_kommune = bosatte_innen_holdeplass.pivot(index="KOMMUNENR_left", columns=["AvstKlasse"],values="population")
bosatte_kommune.head()

AvstKlasse,Innen 500 m,Mer enn 500 m
KOMMUNENR_left,,
3401,8115.00000,489.00000
3414,2169.00000,402.00000
3415,2340.00000,1119.00000
3416,2988.00000,312.00000
3417,2145.00000,588.00000


In [79]:
# Beregne andel
bosatte_kommune["Andel"] = bosatte_kommune["Innen 500 m"] / (bosatte_kommune["Innen 500 m"] + bosatte_kommune["Mer enn 500 m"]) 

In [80]:
bosatte_kommune.head()

AvstKlasse,Innen 500 m,Mer enn 500 m,Andel
KOMMUNENR_left,,,
3401,8115.00000,489.00000,0.94317
3414,2169.00000,402.00000,0.84364
3415,2340.00000,1119.00000,0.67650
3416,2988.00000,312.00000,0.90545
3417,2145.00000,588.00000,0.78485
